In [3]:
nvidia-smi


NameError: name 'nvidia' is not defined

In [4]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from ultralytics import YOLO
import cv2
import glob
import shutil
import random


# Define paths
train_path = "/kaggle/input/ucf-crime-dataset/Train"
test_path = "/kaggle/input/ucf-crime-dataset/Test"

# Function to load frames and labels
def load_dataset(dataset_path):
    frame_paths = []
    labels = []

    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            for frame_name in os.listdir(class_path):
                frame_paths.append(os.path.join(class_path, frame_name))
                labels.append(class_name)

    return frame_paths, labels

# Load training and testing data
X_train, y_train = load_dataset(train_path)
X_test, y_test = load_dataset(test_path)

# Encode labels as integers
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))
print("Classes:", label_encoder.classes_)

Training samples: 1266345
Testing samples: 111308
Classes: ['Abuse' 'Arrest' 'Arson' 'Assault' 'Burglary' 'Explosion' 'Fighting' 'NormalVideos' 'RoadAccidents' 'Robbery' 'Shooting' 'Shoplifting' 'Stealing' 'Vandalism']


In [3]:
DATASET_PATH = "/kaggle/input/ucf-crime-dataset"
TRAIN_PATH = f"{DATASET_PATH}/Train"
TEST_PATH = f"{DATASET_PATH}/Test"
YOLO_DATASET_PATH = "/kaggle/working/UCF-YOLO"


In [4]:
# Create folders for YOLO dataset
os.makedirs(f"{YOLO_DATASET_PATH}/images/train", exist_ok=True)
os.makedirs(f"{YOLO_DATASET_PATH}/images/val", exist_ok=True)
os.makedirs(f"{YOLO_DATASET_PATH}/labels/train", exist_ok=True)
os.makedirs(f"{YOLO_DATASET_PATH}/labels/val", exist_ok=True)


In [5]:
def copy_images(source_folder, dest_folder, train_ratio=0.8):
    all_files = glob.glob(f"{source_folder}/*/*.jpg")  # Adjust if PNG
    random.shuffle(all_files)
    
    train_size = int(len(all_files) * train_ratio)
    
    for i, file in enumerate(all_files):
        filename = os.path.basename(file)
        if i < train_size:
            shutil.copy(file, f"{YOLO_DATASET_PATH}/images/train/{filename}")
        else:
            shutil.copy(file, f"{YOLO_DATASET_PATH}/images/val/{filename}")

# Move frames from train/test folders
copy_images(TRAIN_PATH, f"{YOLO_DATASET_PATH}/images")
copy_images(TEST_PATH, f"{YOLO_DATASET_PATH}/images")


In [5]:
# Load YOLOv8-Pose model
pose_model = YOLO("yolov8n-pose.pt")

# Process images and generate labels
image_folder = f"{YOLO_DATASET_PATH}/images/train"
label_folder = f"{YOLO_DATASET_PATH}/labels/train"

os.makedirs(label_folder, exist_ok=True)

for img_path in glob.glob(f"{image_folder}/*.jpg"):
    results = pose_model(img_path)  # Run inference

    label_file = os.path.join(label_folder, os.path.basename(img_path).replace(".jpg", ".txt"))
    with open(label_file, "w") as f:
        for result in results:
            for pose in result.keypoints.data:
                x_center, y_center = pose[0].item(), pose[1].item()
                f.write(f"0 {x_center} {y_center} 0.1 0.1\n")  # Adjust bounding box size


NameError: name 'YOLO_DATASET_PATH' is not defined

In [6]:
data_yaml_content = """path: /kaggle/working/UCF-YOLO
train: images/train
val: images/val
nc: 1
names: ["person"]
kpt_shape: [17, 3]
"""

yaml_file_path = "/kaggle/working/UCF-YOLO/data.yaml"

with open(yaml_file_path, "w") as file:
    file.write(data_yaml_content)

print("✅ data.yaml updated successfully!")


✅ data.yaml updated successfully!


In [7]:
import os
import glob
import shutil

# Define original dataset paths
DATASET_PATH = "/kaggle/input/ucf-crime-dataset"
TRAIN_PATH = f"{DATASET_PATH}/Train"
TEST_PATH = f"{DATASET_PATH}/Test"

# Define YOLO dataset structure
YOLO_DATASET_PATH = "/kaggle/working/UCF-YOLO"
YOLO_TRAIN_IMAGES = f"{YOLO_DATASET_PATH}/images/train"
YOLO_VAL_IMAGES = f"{YOLO_DATASET_PATH}/images/val"

# Create YOLO dataset directories
os.makedirs(YOLO_TRAIN_IMAGES, exist_ok=True)
os.makedirs(YOLO_VAL_IMAGES, exist_ok=True)

# Define selected categories
SELECTED_CATEGORIES = ["Abuse", "Arson", "Assault", "Fighting", "NormalVideos", "Robbery", "Vandalism"]

# Function to move only selected categories into YOLO train/val folders
def move_selected_images(source_folder, dest_folder, train_ratio=0.8):
    all_images = []
    for category in SELECTED_CATEGORIES:
        category_path = os.path.join(source_folder, category)
        if os.path.exists(category_path):
            all_images.extend(glob.glob(f"{category_path}/*.png"))  # Change to .jpg if needed

    train_size = int(len(all_images) * train_ratio)

    for i, image in enumerate(all_images):
        filename = os.path.basename(image)
        if i < train_size:
            shutil.copy(image, f"{YOLO_TRAIN_IMAGES}/{filename}")
        else:
            shutil.copy(image, f"{YOLO_VAL_IMAGES}/{filename}")

# Move images from Train and Test sets
move_selected_images(TRAIN_PATH, YOLO_TRAIN_IMAGES)
move_selected_images(TEST_PATH, YOLO_VAL_IMAGES)

print("✅ Only selected categories moved successfully!")


✅ Only selected categories moved successfully!


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8n-pose.pt")  # Load YOLOv8 Pose model
model.train(
    data="/kaggle/working/UCF-YOLO/data.yaml",
    epochs=5,
    imgsz=640,
    batch=2,  # Reduce batch size
    workers=2,  # Increase workers to prevent dataloader freezing
    save_period=1  # Save model every epoch
)


Ultralytics 8.3.83 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=pose, mode=train, model=yolov8n-pose.pt, data=/kaggle/working/UCF-YOLO/data.yaml, epochs=5, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning /kaggle/working/UCF-YOLO/labels/train... 0 images, 965654 backgrounds, 1 corrupt: 100%|██████████| 965655/965655 [09:31<00:00, 1689.07it/s]

train: WARNING ⚠️ /kaggle/working/UCF-YOLO/images/train/Arrest026_x264_4150.png: ignoring corrupt image/label: cannot identify image file '/kaggle/working/UCF-YOLO/images/train/Arrest026_x264_4150.png'
train: WARNING ⚠️ No labels found in /kaggle/working/UCF-YOLO/labels/train.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


train: New cache created: /kaggle/working/UCF-YOLO/labels/train.cache
WARNING ⚠️ No labels found in /kaggle/working/UCF-YOLO/labels/train.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
WARNING ⚠️ No 'flip_idx' array defined in data.yaml, setting augmentation 'fliplr=0.0'
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /kaggle/working/UCF-YOLO/labels/val... 0 images, 231062 backgrounds, 0 corrupt: 100%|██████████| 231062/231062 [02:21<00:00, 1629.15it/s]

val: WARNING ⚠️ No labels found in /kaggle/working/UCF-YOLO/labels/val.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


val: New cache created: /kaggle/working/UCF-YOLO/labels/val.cache
WARNING ⚠️ No labels found in /kaggle/working/UCF-YOLO/labels/val.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
Plotting labels to runs/pose/train4/labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/pose/train4
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


        1/5      2.08G          0          0          0      4.407          0          0        640:  17%|█▋        | 10473/60354 [26:52<2:07:13,  6.53it/s]

In [2]:
import cv2

def preprocess_frame(frame_path, target_size=(224, 224)):
    # Load the frame
    frame = cv2.imread(frame_path)
    # Resize the frame
    frame = cv2.resize(frame, target_size)
    # Normalize pixel values to [0, 1]
    frame = frame / 255.0
    return frame

# Example: Preprocess a frame
sample_frame = preprocess_frame(X_train[0])
print("Frame shape:", sample_frame.shape)

Frame shape: (224, 224, 3)


In [3]:
import numpy as np

def data_generator(frame_paths, labels, batch_size=32):
    num_samples = len(frame_paths)
    while True:
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i + batch_size]
            batch_paths = [frame_paths[j] for j in batch_indices]
            batch_labels = [labels[j] for j in batch_indices]
            batch_frames = [preprocess_frame(path) for path in batch_paths]
            yield np.array(batch_frames), np.array(batch_labels)

# Create data generators
batch_size = 32
train_gen = data_generator(X_train, y_train, batch_size)
test_gen = data_generator(X_test, y_test, batch_size)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load pre-trained MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(label_encoder.classes_), activation='softmax')(x)  # Output layer

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_test) // batch_size

history = model.fit(train_gen, steps_per_epoch=steps_per_epoch,
                    validation_data=test_gen, validation_steps=validation_steps,
                    epochs=10)

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
18262/39573 ━━━━━━━━━━━━━━━━━━━━ 2:18:48 391ms/step - accuracy: 0.9443 - loss: 0.2064

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_gen, steps=validation_steps)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
# Save the model
model.save("anomaly_detection_model.h5")

# Convert to TensorFlow Lite (optional)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)